# Evaluation of calssification/segmentation models

This jupyter notebook is used to calculate the f1-score, accuracy, precision and recall. It is based on the segmentation masks and the class label.

In [ ]:
import scipy
import numpy as np
import skimage
import json
import os
import glob as glob
import re

In [ ]:
def extract_bounding_boxes_and_labels(labeled_image, label_dict):
    """_summary_

    Args:
        labeled_image (_type_): labeled image as numpy array
        label_dict (_type_): class dictionary from Qupath

    Returns:
        _type_: bounding boxes of each object and it's label
    """
    objects = scipy.ndimage.find_objects(labeled_image)
    bounding_boxes = []
    labels = []
    
    for obj_id, obj_slice in enumerate(objects):
        if obj_slice is not None:
            ymin, xmin, ymax, xmax = obj_slice[0].start, obj_slice[1].start, obj_slice[0].stop, obj_slice[1].stop
            bounding_boxes.append((xmin, ymin, xmax - xmin, ymax - ymin))  # Format: (x, y, width, height)
            for class_label, ids in label_dict.items():
                if obj_id + 1 in ids:  # labels start from 1
                    labels.append(class_label)
                    break
            else:
                labels.append('unknown')  # If not found in any class
    return bounding_boxes, labels


In [ ]:
def compute_iou(box1, box2):
    """_summary_

    Args:
        box1 (_type_): bounding box of first object
        box2 (_type_): bounding box of second object

    Returns:
        _type_: intersection over union
    """
    
    #get coordinates
    x_left = max(box1[0], box2[0])
    y_top = max(box1[1], box2[1])
    x_right = min(box1[0] + box1[2], box2[0] + box2[2])
    y_bottom = min(box1[1] + box1[3], box2[1] + box2[3])

    #if not overlapping
    if x_right < x_left or y_bottom < y_top:
        return 0.0

    #get intersection
    intersection_area = (x_right - x_left) * (y_bottom - y_top)
    #get area of the boxes
    box1_area = box1[2] * box1[3]
    box2_area = box2[2] * box2[3]
    #calculated the union
    union_area = box1_area + box2_area - intersection_area
    #calculate the iou
    iou = intersection_area / union_area
    return iou

def evaluate_detection(pred_boxes, pred_labels, gt_boxes, gt_labels, iou_threshold=0.5):
    """_summary_

    Args:
        pred_boxes (_type_): bounding box of prediction
        pred_labels (_type_): predicted label
        gt_boxes (_type_): bounding box of ground truth
        gt_labels (_type_): ground truth label
        iou_threshold (float, optional): IoU threshold. Defaults to 0.5.

    Returns:
        _type_: tuple with tp, fp, fn
    """
    tp = 0
    fp = 0
    fn = 0
    matched_gt = []

    for pred_box, pred_label in zip(pred_boxes, pred_labels):
        best_iou = 0
        best_gt_idx = None
        for gt_idx, (gt_box, gt_label) in enumerate(zip(gt_boxes, gt_labels)):
            if gt_idx in matched_gt: #count only once
                continue
            iou = compute_iou(pred_box, gt_box) #compute iou
            if iou > best_iou: #check for best overlap
                best_iou = iou
                best_gt_idx = gt_idx
        
        if best_iou >= iou_threshold and pred_label == gt_labels[best_gt_idx]: #check for iou over threshold and correct label
            tp += 1
            matched_gt.append(best_gt_idx)
        else:
            fp += 1
    
    fn = len(gt_boxes) - len(matched_gt)

    return tp, fp, fn

def aggregate_metrics(results):
    """_summary_

    Args:
        results (_type_): dictionary with tp, fp and fn

    Returns:
        _type_: a dictionary with calculated metrics: tp, fp, fn, f1-score, accuracy, precision and recall
    """
    tp = sum([result['tp'] for result in results])
    fp = sum([result['fp'] for result in results])
    fn = sum([result['fn'] for result in results])

    
    precision = tp / (tp + fp) if tp + fp > 0 else 0
    recall = tp / (tp + fn) if tp + fn > 0 else 0
    accuracy = tp / (tp + fp + fn) if tp + fp + fn > 0 else 0
    f1_score = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0

    return {
        'tp': tp,
        'fp': fp,
        'fn': fn,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score
    }

In [ ]:

#This function does the same as the above but here I also confirm that the scipys order of objects match the actual label id in the image assigned by qupath 
def extract_bounding_boxes_and_labels(labeled_image, label_dict):
    objects = scipy.ndimage.find_objects(labeled_image)
    labeled_image = labeled_image.astype(np.int32)
    bounding_boxes = []
    labels = []
    
    # Invert the label_dict to map object IDs to class labels
    inverted_label_dict = {}
    for class_label, ids in label_dict.items():
        for obj_id in ids:
            inverted_label_dict[obj_id] = class_label
    
    for obj_id, obj_slice in enumerate(objects, 1):
        if obj_slice is not None:
            ymin, xmin, ymax, xmax = obj_slice[0].start, obj_slice[1].start, obj_slice[0].stop, obj_slice[1].stop
            bbox = (xmin, ymin, xmax - xmin, ymax - ymin)  # Format: (x, y, width, height)
            sub_image = labeled_image[ymin:ymax, xmin:xmax]
            unique_ids, counts = np.unique(sub_image, return_counts=True)
            # Filter out background
            mask = unique_ids != 0
            unique_ids = unique_ids[mask]
            counts = counts[mask]
            if len(counts) == 0:
                continue  # Skip if no valid objects
            dominant_id = unique_ids[np.argmax(counts)]
            assert obj_id == dominant_id
            label = inverted_label_dict.get(dominant_id, 'unknown')
            bounding_boxes.append(bbox)
            labels.append(label)
    
    return bounding_boxes, labels

def process_image_folder(pred_folder, gt_folder, i):
    """Process the folder of images to be used for evaluation. 

    Args:
        pred_folder (_type_): folder to predicted masks and classes
        gt_folder (_type_): folder to ground truth masks and classes
        i (_type_): roi number from QuPath (if more than one roi per image, we need to handle these indivdually)

    Returns:
        _type_: summarized metrics
    """
    results = []

    pred_files = glob.glob(os.path.join(pred_folder, f"*{i}-labels.tiff"))
    gt_files = glob.glob(os.path.join(gt_folder, f"*{i}-labels.tiff"))

    for pred_filepath,gt_filepath in zip(pred_files,gt_files):
        # Extract base name without the digit and -labels.tiff
        base_name_match = re.match(r'(.+)-labels\.tiff$', os.path.basename(pred_filepath))

        base_name = base_name_match.group(1)[:-2]

        
        pred_json_path = os.path.join(pred_folder, f"{base_name}-classmap.json")
        gt_json_path = os.path.join(gt_folder, f"{base_name}-classmap.json")
        
        if os.path.exists(gt_filepath) and os.path.exists(pred_json_path) and os.path.exists(gt_json_path):
            pred_labeled_image = skimage.io.imread(pred_filepath)
            gt_labeled_image = skimage.io.imread(gt_filepath)
            
            with open(pred_json_path, 'r') as f:
                pred_label_dict = json.load(f)

            with open(gt_json_path, 'r') as f:
                gt_label_dict = json.load(f)

            pred_boxes, pred_labels = extract_bounding_boxes_and_labels(pred_labeled_image, pred_label_dict)
            gt_boxes, gt_labels = extract_bounding_boxes_and_labels(gt_labeled_image, gt_label_dict)

            tp, fp, fn = evaluate_detection(pred_boxes, pred_labels, gt_boxes, gt_labels)
            results.append({'tp': tp, 'fp': fp, 'fn': fn})

    return aggregate_metrics(results)

cellpose_fineTuned_metrics_dict = dict()
cellpose_unTuned_metrics_dict = dict()
QuPath_RF_metrics_dict = dict()
for i in range(1, 3):
    print(f"Metrics for images with ROIs with nr {i}")
    gt_folder = "Ground_Truth folder"
    pred_folder = "Cellpose_FineTuned folder"
    cellpose_fineTuned_metrics_dict[i] = process_image_folder(pred_folder, gt_folder,i)
    print(f"Cellpose  finetuned: {cellpose_fineTuned_metrics_dict[i]}")


    pred_folder = "Cellpose_UnTuned folder"
    cellpose_unTuned_metrics_dict[i] = process_image_folder(pred_folder, gt_folder,i)
    print(f"Cellpose  Untuned: {cellpose_unTuned_metrics_dict[i]}")

    pred_folder = "QuPath_RF folder"
    QuPath_RF_metrics_dict[i] = process_image_folder(pred_folder, gt_folder,i)
    print(f"QuPath RF: {QuPath_RF_metrics_dict[i]}")

The rest of the code is just to combine the results from all the images and save the output accordingly 

In [ ]:
final_dict = dict()
def combine_dicts(dict1, dict2):
    combined_dict = {}

    # Iterate through the keys in the first dictionary
    for key in dict1:
        if key in dict2:
            combined_dict[key] = dict1[key] + dict2[key]
        else:
            combined_dict[key] = dict1[key]

    # Add the keys from the second dictionary that are not in the first dictionary
    for key in dict2:
        if key not in combined_dict:
            combined_dict[key] = dict2[key]

    combined_dict['accuracy'] = combined_dict['accuracy'] / 2 
    combined_dict['precision'] = combined_dict['precision'] / 2 
    combined_dict['recall'] = combined_dict['recall'] / 2 
    combined_dict['f1_score'] = combined_dict['f1_score'] / 2 
    return combined_dict

final_dict["Cellpose_FineTuned"] = combine_dicts(cellpose_fineTuned_metrics_dict[1], cellpose_fineTuned_metrics_dict[2])
final_dict["Cellpose_UnTuned"] = combine_dicts(cellpose_unTuned_metrics_dict[1], cellpose_unTuned_metrics_dict[2])
final_dict["QuPath_RF"] = combine_dicts(QuPath_RF_metrics_dict[1], QuPath_RF_metrics_dict[2])
print(final_dict) 

In [ ]:
import csv

def save_nested_dict_to_csv(nested_dict, csv_filename):
    # Extract fieldnames from nested dictionaries
    fieldnames = set()
    for key, sub_dict in nested_dict.items():
        fieldnames.update(sub_dict.keys())

    # Add the top-level keys to the fieldnames
    fieldnames = ['Method'] + list(fieldnames)

    # Flatten the nested dictionary
    rows = []
    for key, sub_dict in nested_dict.items():
        row = {'Method': key}
        row.update(sub_dict)
        rows.append(row)

    # Write to CSV file
    with open(csv_filename, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(rows)
        
# Save to CSV
outpath = "<path to output>"
save_nested_dict_to_csv(final_dict, outpath)